# Scrap compound on PDBe foreach molecule
**18.01.22 - Third week of internship**
### 1. Imports

In [4]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import csv
import html5lib
import json

import pyspark

from pyspark.sql.functions import (
    col, udf, struct, lit, split, regexp_replace, create_map, min as spark_min, max as spark_max,
    count, sum as spar_sum, explode, when, concat, lower, format_string, collect_set, collect_list
)

from pyspark.sql.types import (
    FloatType, ArrayType, StructType, StructField, StringType, IntegerType, TimestampType
)

from pyspark.sql import SparkSession, Row
from pyspark.conf import SparkConf
from itertools import chain
from pyspark import SparkFiles

# for heatmap:
import numpy as np
import seaborn as sns
import matplotlib.pylab as plt
import os


### 2. Create a Spark session

In [5]:
spark = SparkSession.builder.appName('abc').getOrCreate()

### 3. Script to construct a disctionary with molecule and corresponding compound
Marine version

In [8]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.61 Safari/537.36'}
pdb_unichem_mol_path = '/Users/marinegirardey/Documents/opentarget_internship/OpenTargetInternship/second_week/id_molecule_id_unichem_merged.csv'
it_header = 0
dict_list = []
list_of_dict = []

with open(pdb_unichem_mol_path) as pdb_unichem_mol:

    for molecule in pdb_unichem_mol:

        if it_header != 0:

            pdb_id = molecule.strip('\n').split(',')[2]
            url='https://www.ebi.ac.uk/pdbe/api/pdb/compound/in_pdb/' + pdb_id
            r=requests.get(url, headers=headers, timeout=2)
            soup = BeautifulSoup(r.content, 'html.parser')
            pretty_soup = soup.prettify()
            id_dict = json.loads(pretty_soup)
            list_of_dict.append(id_dict)

        it_header += 1

Daniel version

In [12]:
import requests

def get_pdb(code: str)-> str:
    URL = f'https://www.ebi.ac.uk/pdbe/api/pdb/compound/in_pdb/{code}'
    response = requests.get(URL)
    try:
        data = response.json()
        return data[code]
    except:
        return None

### 4. Function to obtain compound from a given molecule
(Tests)

In [15]:
# Construct a list of dict with pdb molecule id
dict_list_only_mol_id = []
for mol_dict in list_of_dict:
    for key, val in mol_dict.items():
        dict_list_only_mol_id.append({'id': key})

In [10]:
# Function to apply on the pd df to get the compouds of each molecule
def get_pdb2(mol_id):
    for mol_dict in list_of_dict:
        for key, val in mol_dict.items():
            if key == mol_id:
                compound = list(val)
    return compound

In [18]:
# Create a df with pdb molecule id and compouds in an array style
(
    spark.createDataFrame(dict_list_only_mol_id)
    .toPandas()
    .assign(pdb = lambda x: x['id'].apply(get_pdb2))
)

,id,pdb
0,ABZ,"[1s9g, 6qbb]"
1,SG2,[1kwr]
2,DOB,"[1dod, 1doe, 1pbb]"
3,UEG,"[4c5l, 4c5n, 5eb3, 6k91]"
4,PZA,"[3r4x, 3r55, 5fpd, 5rtm]"
...,...,...
3640,HCS,"[1jqw, 1jvi, 1q8a, 1q8j, 1u22, 1xdj, 2ci5, 2lp..."
3641,BGC,"[1aa5, 1abr, 1au1, 1ax1, 1b1y, 1b2y, 1bag, 1bg..."
3642,PGA,"[1a3w, 1a3x, 1amk, 1aw1, 1btm, 1egh, 1g5k, 1g5..."
3643,ALR,[1az1]
